# Ensemble Retriever `Convex Combination(CC)` 추가

[written by@teddynote](https://github.com/teddylee777/langchain-teddynote)

- 참고글: [AutoRAG 가 게재한 알고리즘 방식의 차이 설명](https://velog.io/@autorag/%EB%9E%AD%EC%B2%B4%EC%9D%B8%EC%9D%98-Ensemble-Retriever-%EC%9D%B4%EA%B2%8C-%EB%8C%80%EC%B2%B4-%EB%AD%90%EC%A7%80)

아래의 주석을 풀고 패키지를 업데이트 후 진행합니다.

In [1]:
# 업데이트 후 진행
!pip install -qU langchain-teddynote

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 587.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""

os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "10-11"

## 실험을 위한 사전 셋업

In [3]:
!pip install -qU langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00


In [5]:
!pip install -qU pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.2 MB/s eta 0:00:00


In [8]:
!pip install -qU faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 34.9 MB/s eta 0:00:00


In [9]:
from langchain.retrievers import EnsembleRetriever as OriginalEnsembleRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_teddynote.retrievers import KiwiBM25Retriever

# 문서 로드(Load Documents)
loader = PDFPlumberLoader("/content/data/디지털 정부혁신 추진계획.pdf")

# 문서 분할(Split Documents): 테스트를 위하여 작은 Chunk Size로 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
split_documents = loader.load_and_split(text_splitter)

# 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# FaissRetriever 생성
faiss = FAISS.from_documents(
    documents=split_documents, embedding=embeddings
).as_retriever(search_kwargs={"k": 5})

# KiwiBM25Retriever 생성(한글 형태소 분석기 + BM25 알고리즘)
bm25 = KiwiBM25Retriever.from_documents(documents=split_documents, embedding=embeddings)
bm25.k = 5

# LangChain 버전의 EnsembleRetriever
original_ensemble_retriever = OriginalEnsembleRetriever(retrievers=[faiss, bm25])

CC 방식과 RRF 방식의 EnsembleRetriever 생성

In [10]:
from langchain_teddynote.retrievers import (
    EnsembleRetriever,
    EnsembleMethod,
)

# RRF 방식의 EnsembleRetriever (기본값으로 RRF 가 설정되어 있음)
rrf_ensemble_retriever = EnsembleRetriever(
    retrievers=[faiss, bm25], method=EnsembleMethod.RRF
)

# CC 방식의 EnsembleRetriever
cc_ensemble_retriever = EnsembleRetriever(
    retrievers=[faiss, bm25], method=EnsembleMethod.CC  # method 지정: CC
)

## 검색 결과 비교

In [11]:
def pretty_print(query):
    for i, (original_doc, cc_doc, rrf_doc) in enumerate(
        zip(
            original_ensemble_retriever.invoke(query),
            cc_ensemble_retriever.invoke(query),
            rrf_ensemble_retriever.invoke(query),
        )
    ):
        print(f"[{i}] [Original] Q: {query}", end="\n\n")
        print(original_doc.page_content)
        print("-" * 100)
        print(f"[{i}] [RRF] Q: {query}", end="\n\n")
        print(rrf_doc.page_content)
        print("-" * 100)
        print(f"[{i}] [CC] Q: {query}", end="\n\n")
        print(cc_doc.page_content)
        print("=" * 100, end="\n\n")

- 검색 결과에 `"Original"` 과 `"RRF"` 는 차이가 없어야 합니다. (LangChain 그대로 구현)
- 검색 결과에 `"CC"` 는 `"RRF"` 와 차이가 있을 수 있습니다.

`RRF` 와 `CC` 방식의 검색 결과 비교하여 문서에 적합한 방식을 차용하시길 바랍니다.

In [12]:
# 검색 결과 비교
pretty_print("디지털 트랜스포메이션이란 무엇인가요?")

[0] [Original] Q: 디지털 트랜스포메이션이란 무엇인가요?

디지털기기 사용이 어려운 분들이 차별없이 서비스를 받도록 지원하겠습니.다
권익위,
□1 국민의 소리 청취·분석 시스템 개선 (22년)
각부처·지자체
----------------------------------------------------------------------------------------------------
[0] [RRF] Q: 디지털 트랜스포메이션이란 무엇인가요?

디지털기기 사용이 어려운 분들이 차별없이 서비스를 받도록 지원하겠습니.다
권익위,
□1 국민의 소리 청취·분석 시스템 개선 (22년)
각부처·지자체
----------------------------------------------------------------------------------------------------
[0] [CC] Q: 디지털 트랜스포메이션이란 무엇인가요?

○ (시스템) 디지털 신기술의 적기 도입과 활용 곤란
- 기존 복잡한 용역개발 방식은 혁신주기가 짧은 디지털 전환에 부적합

[1] [Original] Q: 디지털 트랜스포메이션이란 무엇인가요?

○ (시스템) 디지털 신기술의 적기 도입과 활용 곤란
- 기존 복잡한 용역개발 방식은 혁신주기가 짧은 디지털 전환에 부적합
----------------------------------------------------------------------------------------------------
[1] [RRF] Q: 디지털 트랜스포메이션이란 무엇인가요?

○ (시스템) 디지털 신기술의 적기 도입과 활용 곤란
- 기존 복잡한 용역개발 방식은 혁신주기가 짧은 디지털 전환에 부적합
----------------------------------------------------------------------------------------------------
[1] [CC] Q: 디지털 트랜스포메이션이란 무엇인가요?

디지털기